In [1]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [2]:
output_model_file = 'posnega_model1/pytorch_model.bin'
output_config_file = 'posnega_model1/config.json'
output_vocab_file = 'posnega_model1/vocab.txt'

config = BertConfig.from_json_file(output_config_file)
model = BertForSequenceClassification(config)
state_dict = torch.load(output_model_file)
model.load_state_dict(state_dict)
tokenizer = BertTokenizer(output_vocab_file, do_lower_case=False)

In [3]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [4]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1080 Ti


In [5]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [6]:
model = model.to(device)
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(torch.tensor(b_input_ids).to(device).long(), 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [7]:
logits = test_sentences(['완봉승을 기록한 워윅 서폴드의 공이 가장 컸다'])

print(logits)
print(np.argmax(logits))

C:\Users\DeepLearning_1\anaconda3\envs\windup\lib\site-packages\ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


[[-3.2392924  3.5598252]]
1


In [8]:
# fine-tuning
train = pd.read_csv('finetune1.csv', encoding='cp949')
train

,Unnamed: 0,sentence,label
0,0.0,“대구 힘내세요” “기억할게요”…희망 주는 ‘야구’가 돌아왔다ㆍ무관중 개막 삼성 -...,1.0
1,1.0,"그럼에도 불구하고, 개막을 오래 기다린 SK팬이라면 허망할 수밖에 없는 완패였다",0.0
2,2.0,어린이들이 가득 차야 할 대구 삼성라이온즈파크 외야 관중석에는 ‘힘내라 대한민국 ...,1.0
3,3.0,마지막까지 힘을 모아 함께 극복합시다!’라는 대형 현수막이 걸렸다,1.0
4,4.0,연고팀 삼성이 코로나19로 어려움을 겪은 대구·경북 주민들을 향해 보내는 기원 메...,1.0
...,...,...,...
25284,25550.0,타선지원이 아쉬웠다,0.0
25285,25551.0,1-0으로 앞선 7회초 선두 타자 사구와 볼넷에 이어 번트 수비를 하다 3루에 ...,0.0
25286,25552.0,두번째 투수 최지광이 실점하면서 최채흥의 승리는 물거품이 됐다,1.0
25287,25553.0,하지만 이날 최채흥의 경기 운영 능력은 만점에 가까웠다,1.0


In [9]:
train = train.dropna(axis=0)
train

,Unnamed: 0,sentence,label
0,0.0,“대구 힘내세요” “기억할게요”…희망 주는 ‘야구’가 돌아왔다ㆍ무관중 개막 삼성 -...,1.0
1,1.0,"그럼에도 불구하고, 개막을 오래 기다린 SK팬이라면 허망할 수밖에 없는 완패였다",0.0
2,2.0,어린이들이 가득 차야 할 대구 삼성라이온즈파크 외야 관중석에는 ‘힘내라 대한민국 ...,1.0
3,3.0,마지막까지 힘을 모아 함께 극복합시다!’라는 대형 현수막이 걸렸다,1.0
4,4.0,연고팀 삼성이 코로나19로 어려움을 겪은 대구·경북 주민들을 향해 보내는 기원 메...,1.0
...,...,...,...
25284,25550.0,타선지원이 아쉬웠다,0.0
25285,25551.0,1-0으로 앞선 7회초 선두 타자 사구와 볼넷에 이어 번트 수비를 하다 3루에 ...,0.0
25286,25552.0,두번째 투수 최지광이 실점하면서 최채흥의 승리는 물거품이 됐다,1.0
25287,25553.0,하지만 이날 최채흥의 경기 운영 능력은 만점에 가까웠다,1.0


In [10]:
train = train.astype({'label':'int'})
train.dtypes

Unnamed: 0    float64
sentence       object
label           int32
dtype: object

In [11]:
# 리뷰 문장 추출
sentences = train['sentence']
sentences[:10]

0    “대구 힘내세요” “기억할게요”…희망 주는 ‘야구’가 돌아왔다ㆍ무관중 개막 삼성 -...
1         그럼에도 불구하고, 개막을 오래 기다린 SK팬이라면 허망할 수밖에 없는 완패였다
2     어린이들이 가득 차야 할 대구 삼성라이온즈파크 외야 관중석에는 ‘힘내라 대한민국 ...
3                 마지막까지 힘을 모아 함께 극복합시다!’라는 대형 현수막이 걸렸다
4     연고팀 삼성이 코로나19로 어려움을 겪은 대구·경북 주민들을 향해 보내는 기원 메...
5      지난 2월 일본 오키나와에서 스프링캠프를 치를 때까지만 해도 삼성은 야구를 꿈꾸...
6                   대구·경북 지역은 코로나19 확진자가 가장 많이 나온 곳이었다
7                   삼성은 일본의 입국 제한 조치 여파로 3월7일 급히 짐을 쌌다
8                 귀국 뒤에도 철저한 규제 속에서 고립된 채 조용히 시즌을 준비했다
9      수많은 이들의 노력 끝에 코로나19 사태가 진정세에 접어들면서 올 것 같지 않았...
Name: sentence, dtype: object

In [12]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] “대구 힘내세요” “기억할게요”…희망 주는 ‘야구’가 돌아왔다ㆍ무관중 개막 삼성 - NC전 관중석엔 “함께 극복을” 현수막 ㆍ코로나19 의료진 응원 영상에 선수들도 ‘덕분에’ 손동작 ㆍ각 구장 깜짝 시구·재치있는 플래카드 ‘야구의 계절’ 환영  관중은 없어도 오늘은 좋은 날 SK 치어리더들이 5일 인천 SK행복드림구장에서 열린 한화와의 KBO리그 개막전에서 온라인으로 생중계되는 응원을 펼치고 있다 [SEP]',
 '[CLS] 그럼에도 불구하고, 개막을 오래 기다린 SK팬이라면 허망할 수밖에 없는 완패였다 [SEP]',
 '[CLS]  어린이들이 가득 차야 할 대구 삼성라이온즈파크 외야 관중석에는 ‘힘내라 대한민국 그동안 수고하셨습니다 [SEP]',
 '[CLS]  마지막까지 힘을 모아 함께 극복합시다!’라는 대형 현수막이 걸렸다 [SEP]',
 '[CLS]  연고팀 삼성이 코로나19로 어려움을 겪은 대구·경북 주민들을 향해 보내는 기원 메시지였다 [SEP]',
 '[CLS]   지난 2월 일본 오키나와에서 스프링캠프를 치를 때까지만 해도 삼성은 야구를 꿈꾸기 어려웠다 [SEP]',
 '[CLS]  대구·경북 지역은 코로나19 확진자가 가장 많이 나온 곳이었다 [SEP]',
 '[CLS]  삼성은 일본의 입국 제한 조치 여파로 3월7일 급히 짐을 쌌다 [SEP]',
 '[CLS]  귀국 뒤에도 철저한 규제 속에서 고립된 채 조용히 시즌을 준비했다 [SEP]',
 '[CLS]   수많은 이들의 노력 끝에 코로나19 사태가 진정세에 접어들면서 올 것 같지 않았던 개막이 다가왔다 [SEP]']

In [13]:
# 라벨 추출
labels = train['label'].values
labels

array([1, 0, 1, ..., 1, 1, 0])

In [14]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] “대구 힘내세요” “기억할게요”…희망 주는 ‘야구’가 돌아왔다ㆍ무관중 개막 삼성 - NC전 관중석엔 “함께 극복을” 현수막 ㆍ코로나19 의료진 응원 영상에 선수들도 ‘덕분에’ 손동작 ㆍ각 구장 깜짝 시구·재치있는 플래카드 ‘야구의 계절’ 환영  관중은 없어도 오늘은 좋은 날 SK 치어리더들이 5일 인천 SK행복드림구장에서 열린 한화와의 KBO리그 개막전에서 온라인으로 생중계되는 응원을 펼치고 있다 [SEP]
['[CLS]', '[UNK]', '대', '##구', '힘', '##내', '##세', '##요', '[UNK]', '[UNK]', '기', '##억', '##할', '##게', '##요', '[UNK]', '[UNK]', '희', '##망', '주', '##는', '[UNK]', '야구', '[UNK]', '가', '돌', '##아', '##왔다', '##ㆍ', '##무', '##관', '##중', '개', '##막', '삼', '##성', '-', 'NC', '##전', '관', '##중', '##석', '##엔', '[UNK]', '함께', '극', '##복', '##을', '[UNK]', '현', '##수', '##막', 'ㆍ', '##코', '##로', '##나', '##19', '의', '##료', '##진', '응', '##원', '영', '##상', '##에', '선수', '##들', '##도', '[UNK]', '덕', '##분에', '[UNK]', '손', '##동', '##작', 'ㆍ', '##각', '구', '##장', '깜', '##짝', '시', '##구', '·', '재', '##치', '##있는', '플', '##래', '##카', '##드', '[UNK]', '야구', '##의', '계', '##절', '[UNK]', '환', '##영', '관', '##중', '##은', '없', '##어', '##도', '오', '##늘', '##은', '좋은', '날', 'SK', '치', '##어', '##리', '##더',

In [15]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,    100,   9069,  17196,  10028,  31605,  24982,  48549,
          100,    100,   8932,  91837,  14843,  14153,  48549,    100,
          100,  10023,  89292,   9689,  11018,    100, 106603,    100,
         8843,   9091,  16985,  72995, 111836,  32537,  20595,  41693,
         8857, 118907,   9410,  17138,    118,  55838,  16617,   8900,
        41693,  40958,  86933,    100,  19653,   8925,  70915,  10622,
          100,   9978,  15891, 118907,   2070,  25517,  11261,  16439,
        54055,   9637,  38688,  18623,   9636,  14279,   9574,  14871,
        10530,  78930,  27023,  12092,    100,   9075, 110355,    100,
         9450,  18778,  38709,   2070,  66540,   8908,  13890,   8943,
       119236,   9485,  17196,    217,   9659,  18622,  84177,   9944,
        37388,  24206,  15001,    100, 106603,  10459,   8887,  58931,
          100,   9995,  30858,   8900,  41693,  10892,   9555,  12965,
        12092,   9580, 118762,  10892,  79633,   8985,  21275,   9779,
      

In [16]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [17]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([   101,   9484,  12692,   9995,  20309,  12178,   9410,  17138,  39900,
         18392,   9580,  31531,  48253,   8908,  11261,  17196,   8888, 119259,
          9477,  24206,  10739, 118795,  11489,  66421,  23607, 102574,  21711,
         17196,   9410,  17138,   9157,  10739,  37093,  24891,  12638,   9838,
        119169,  10025,  81483,  24891,  10459,  81785,   9484,  12692,  11102,
          9410,  17138,  78930,  20173,  86015,  10003,   9995,  20309,  12453,
         11506,    102,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [18]:
# 배치 사이즈
batch_size = 8

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [19]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [20]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [21]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [24]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
torch.backends.cudnn.enabled = False

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
       
        # Forward 수행                
        outputs = model(torch.tensor(b_input_ids).to(device).long(), 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels.long())
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(torch.tensor(b_input_ids).to(device).long(), 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...


C:\Users\DeepLearning_1\anaconda3\envs\windup\lib\site-packages\ipykernel_launcher.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


  Batch   500  of  2,337.    Elapsed: 0:01:13.
  Batch 1,000  of  2,337.    Elapsed: 0:02:26.
  Batch 1,500  of  2,337.    Elapsed: 0:03:39.
  Batch 2,000  of  2,337.    Elapsed: 0:04:52.

  Average training loss: 0.38
  Training epcoh took: 0:05:41

Running Validation...


C:\Users\DeepLearning_1\anaconda3\envs\windup\lib\site-packages\ipykernel_launcher.py:107: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


  Accuracy: 0.87
  Validation took: 0:00:08

======== Epoch 2 / 4 ========
Training...
  Batch   500  of  2,337.    Elapsed: 0:01:13.
  Batch 1,000  of  2,337.    Elapsed: 0:02:25.
  Batch 1,500  of  2,337.    Elapsed: 0:03:38.
  Batch 2,000  of  2,337.    Elapsed: 0:04:51.

  Average training loss: 0.29
  Training epcoh took: 0:05:39

Running Validation...
  Accuracy: 0.89
  Validation took: 0:00:08

======== Epoch 3 / 4 ========
Training...
  Batch   500  of  2,337.    Elapsed: 0:01:12.
  Batch 1,000  of  2,337.    Elapsed: 0:02:24.
  Batch 1,500  of  2,337.    Elapsed: 0:03:36.
  Batch 2,000  of  2,337.    Elapsed: 0:04:48.

  Average training loss: 0.21
  Training epcoh took: 0:05:36

Running Validation...
  Accuracy: 0.90
  Validation took: 0:00:08

======== Epoch 4 / 4 ========
Training...
  Batch   500  of  2,337.    Elapsed: 0:01:12.
  Batch 1,000  of  2,337.    Elapsed: 0:02:23.
  Batch 1,500  of  2,337.    Elapsed: 0:03:35.
  Batch 2,000  of  2,337.    Elapsed: 0:04:46.

  Av

In [25]:
model = model.to(device)
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(torch.tensor(b_input_ids).to(device).long(), 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [38]:
logits = test_sentences(['맷 윌리엄스 감독은 손톱 상태가 투구가 어렵다고 판단하고 박준표를 대체 투수로 올렸다.'])

print(logits)
print(np.argmax(logits))

[[ 2.8803    -3.1308494]]
0


C:\Users\DeepLearning_1\anaconda3\envs\windup\lib\site-packages\ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [60]:
article = '[OSEN=광주. 이선호 기자] KIA 타이거즈 좌완 이의리가 투구도중 손톱이 깨쳐 조기 강판했다. 이의리는 12일 광주-기아 챔피언스필드에서 열린 NC 다이노스와의 경기에 선발등판해 3이닝을 마치고 등판을 마감했다. 투구성적은 4피안타 3볼넷 5탈삼진 1실점이었다. 0-1로 뒤진 가운데 내려가 시즌 5승 사냥은 불발에 그쳤다. 6경기 연속 승리를 거두지 못했다. 이의리의 강판 이유는 왼손 중지 손톱이 깨지는 부상 때문이었다. 맷 윌리엄스 감독은 손톱 상태가 투구가 어렵다고 판단하고 박준표를 대체 투수로 올렸다. 이의리는 초반부터 많은 투구로 고전했다. 1회초 최정원 안타, 정현 볼넷을 내주고 무사 1,2루에 몰렸다. 세 타자를 삼진 2개와 범타로 막았으나 22구를 던졌다. 2회는 1안타 무실점으로 넘겼으나 3회 제구가 흔들렸다. 1사후 정현 볼넷에 이어 2사후 양의지를 볼넷을 허용하고 알테어에게 중전적시타를 맞고 첫 실점했다. 전민수를 2루 뜬공으로 잡고 이닝을 마감했다. 3회까지 9개의 아웃카운트를 잡는데 69구를 던졌다. 스트라이크는 38개에 그쳤다. 추후 등판 일정은 손톱의 상태에 따라 유동적이다.'
sentences = article.split('.')[:-1]
sentences

['[OSEN=광주',
 ' 이선호 기자] KIA 타이거즈 좌완 이의리가 투구도중 손톱이 깨쳐 조기 강판했다',
 ' 이의리는 12일 광주-기아 챔피언스필드에서 열린 NC 다이노스와의 경기에 선발등판해 3이닝을 마치고 등판을 마감했다',
 ' 투구성적은 4피안타 3볼넷 5탈삼진 1실점이었다',
 ' 0-1로 뒤진 가운데 내려가 시즌 5승 사냥은 불발에 그쳤다',
 ' 6경기 연속 승리를 거두지 못했다',
 ' 이의리의 강판 이유는 왼손 중지 손톱이 깨지는 부상 때문이었다',
 ' 맷 윌리엄스 감독은 손톱 상태가 투구가\xa0어렵다고 판단하고 박준표를 대체 투수로 올렸다',
 ' 이의리는 초반부터 많은 투구로 고전했다',
 ' 1회초 최정원 안타, 정현 볼넷을 내주고 무사 1,2루에 몰렸다',
 ' 세 타자를 삼진 2개와 범타로 막았으나 22구를 던졌다',
 ' 2회는 1안타 무실점으로 넘겼으나 3회 제구가 흔들렸다',
 ' 1사후 정현\xa0볼넷에 이어\xa02사후 양의지를 볼넷을 허용하고 알테어에게 중전적시타를 맞고 첫 실점했다',
 ' 전민수를 2루 뜬공으로 잡고 이닝을 마감했다',
 ' 3회까지 9개의 아웃카운트를 잡는데 69구를 던졌다',
 ' 스트라이크는 38개에 그쳤다',
 ' 추후 등판 일정은 손톱의 상태에 따라 유동적이다']

In [61]:
negative = []
positive = []
myjudge = []
for s in sentences:
#     if(np.argmax(test_sentences([s])) == 1):
#         positive.append('*')
#     else:
#         negative.append('*')
    
    print(s, 'label: ', np.argmax(test_sentences([s])))
    judge = int(input('myjudge? '))
    myjudge.append(judge)
    if judge == 1:
        positive.append('*')
    else:
        negative.append('*')
    
if len(positive) > len(negative):
    print('이 기사는 긍정')
elif len(positive) < len(negative):
    print('이 기사는 부정')
else:
    print('이 기사는 중립')

C:\Users\DeepLearning_1\anaconda3\envs\windup\lib\site-packages\ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


[OSEN=광주 label:  1
myjudge? 0
 이선호 기자] KIA 타이거즈 좌완 이의리가 투구도중 손톱이 깨쳐 조기 강판했다 label:  0
myjudge? 0
 이의리는 12일 광주-기아 챔피언스필드에서 열린 NC 다이노스와의 경기에 선발등판해 3이닝을 마치고 등판을 마감했다 label:  0
myjudge? 0
 투구성적은 4피안타 3볼넷 5탈삼진 1실점이었다 label:  1
myjudge? 1
 0-1로 뒤진 가운데 내려가 시즌 5승 사냥은 불발에 그쳤다 label:  0
myjudge? 0
 6경기 연속 승리를 거두지 못했다 label:  0
myjudge? 0
 이의리의 강판 이유는 왼손 중지 손톱이 깨지는 부상 때문이었다 label:  0
myjudge? 0
 맷 윌리엄스 감독은 손톱 상태가 투구가 어렵다고 판단하고 박준표를 대체 투수로 올렸다 label:  0
myjudge? 0
 이의리는 초반부터 많은 투구로 고전했다 label:  1
myjudge? 0
 1회초 최정원 안타, 정현 볼넷을 내주고 무사 1,2루에 몰렸다 label:  0
myjudge? 0
 세 타자를 삼진 2개와 범타로 막았으나 22구를 던졌다 label:  1
myjudge? 0
 2회는 1안타 무실점으로 넘겼으나 3회 제구가 흔들렸다 label:  0
myjudge? 0
 1사후 정현 볼넷에 이어 2사후 양의지를 볼넷을 허용하고 알테어에게 중전적시타를 맞고 첫 실점했다 label:  0
myjudge? 0
 전민수를 2루 뜬공으로 잡고 이닝을 마감했다 label:  0
myjudge? 1
 3회까지 9개의 아웃카운트를 잡는데 69구를 던졌다 label:  0
myjudge? 0
 스트라이크는 38개에 그쳤다 label:  0
myjudge? 0
 추후 등판 일정은 손톱의 상태에 따라 유동적이다 label:  0
myjudge? 0
이 기사는 부정


In [62]:
f = open(finetune)

In [ ]:
# fine-tuning test
test= pd.read_csv('dataset/finetuning_aihub_model.csv', encoding='cp949')
test

In [ ]:
test = test.dropna(axis=0)
test

In [ ]:
test = test.astype({'label':'int'})
test.dtypes

In [ ]:
# 리뷰 문장 추출
sentences = test['sentence']
sentences[:10]

In [ ]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

In [ ]:
# 라벨 추출
labels = test['label'].values
labels

In [ ]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

In [ ]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

In [ ]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

In [ ]:
# 배치 사이즈
batch_size = 16

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(torch.tensor(b_input_ids).to(device).long(),
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

In [ ]:
logits = test_sentences(['아내가 드디어 출산하게 되어서 정말 신이 나.'])

print(logits)
print(np.argmax(logits))

In [ ]:
sentences = test['sentence']
sentences

In [ ]:
label = []
for s in sentences:
    label.append(np.argmax(test_sentences([s])))
    print(s, 'label: ', np.argmax(test_sentences([s])))

In [ ]:
print(len(label))
print(len(sentences))

In [ ]:
samsung = {'sentence': sentences, 'label': label}
samsungdf = pd.DataFrame(samsung)
samsungdf

In [ ]:
samsungdf.to_csv('finetune1.csv')

In [70]:
model.save_pretrained('posnega_model2')
tokenizer.save_pretrained('posnega_model2')

('posnega_model2\\tokenizer_config.json',
 'posnega_model2\\special_tokens_map.json',
 'posnega_model2\\vocab.txt',
 'posnega_model2\\added_tokens.json')